In [1]:
import dearpygui.dearpygui as dpg
import numpy as np
from math import sqrt

flag = 0

# функция, вызывающаяся при нажатии кнопки
# если на кнопке написано "done" - просчитывает значения и строит график
# если "clear" - очищает все поля
def fun():
    global flag
    
    if flag==0:
        weight = dpg.get_value(wei)
        v0 = sqrt((2*(43639/400000+365197/5000000-0.0435*1/1000*(51.4*5/6+weight*1000)*9.8))/(1/1000*(51.4*5/6+weight*1000)))
        t_raise = -v0*0.13/-9.8
        d_h = v0*0.13*t_raise - (9.8*t_raise*t_raise)/2
        t_fall = (2*(d_h+0.15)/9.8)**0.5
        t_full = round(t_raise,2) + round(t_fall,2)
        dpg.set_value("mh", f"Maximum Flight Altitude: {round(d_h+0.15,2)} m")
        dpg.set_value("tmh", f"Time To Reach Maximum Flight Altitude: {round(t_raise,2)} s")
        dpg.set_value("lmh", f"Flight Time: {round(t_full,2)} s")
        dpg.set_value("lf", f"Flight Distance: {round(v0*0.99*t_full,2)} m")

        l = np.linspace(0, round(v0*0.99*t_full)+1, num=100)
        xl, yl = [], []
        
        for k in l:
            xl.append(v0*0.99*k)
            yl.append(0.15+ v0*0.13*k - (9.8*k*k)/2)
            
        dpg.add_line_series(xl, yl, parent="y_axis", show = True, tag = 'traectory')
        dpg.set_axis_limits("x_axis", 0, round(v0*0.99*t_full,2)*1.5)
        dpg.set_axis_limits("y_axis", 0, round(d_h+0.15,2)*1.5)
        dpg.set_item_label("button", "Clear")
        flag=1
        
    else:
        dpg.set_value("mh", f"Maximum Flight Altitude: ")
        dpg.set_value("tmh", f"Time To Reach Maximum Flight Altitude:")
        dpg.set_value("lmh", f"Flight Time: ")
        dpg.set_value("lf", f"Flight Distance: ")
        dpg.set_item_label("button", "Done")
        dpg.delete_item("traectory")
        flag=0

dpg.create_context()
dpg.create_viewport(x_pos=450, y_pos = 0, max_width = 716, max_height = 800, resizable = False)
dpg.setup_dearpygui()

# установка шрифтов
with dpg.font_registry():
    default_font = dpg.add_font("BankaDemoVersion.otf", 25)
    plot_font = dpg.add_font("BankaDemoVersion.otf", 20)
    second_font = dpg.add_font("China.ttf", 35)

# настройка цветов у текста, кнопки и графика
with dpg.theme() as global_theme:
    with dpg.theme_component(dpg.mvAll):
        dpg.add_theme_color(dpg.mvThemeCol_WindowBg, (0, 0, 0))
        dpg.add_theme_color(dpg.mvPlotCol_Line, (255, 0, 0), category=dpg.mvThemeCat_Plots)
        dpg.add_theme_color(dpg.mvThemeCol_TitleBgActive, (100, 0, 0))
        dpg.add_theme_color(dpg.mvThemeCol_ModalWindowDimBg, (0, 0, 0))
        dpg.add_theme_color(dpg.mvThemeCol_FrameBg, (10, 10, 10))
        dpg.add_theme_color(dpg.mvThemeCol_Button, (100, 0, 0))
        
dpg.bind_theme(global_theme)

width1, height1, channels, data = dpg.load_image("670445.png")
with dpg.texture_registry():
    texture_id = dpg.add_static_texture(width1, height1, data)

# размещение элементов в окне
with dpg.window(label="",width = 716, height = 1000,pos=(0,0), no_close = True, no_resize = True):
    global flag
    dpg.add_image(texture_id, pos=(5,470))
    
    b2 = dpg.add_text("Catapult")
    b3 = dpg.add_text("Calculation")
    dpg.add_text("")
    dpg.add_text("Enter the ball's weight to get")
    dpg.add_text("information about the flight:")
    wei = dpg.add_input_float(label = "kg", default_value=0, min_clamped = True, max_clamped = True, min_value = 0.0, max_value = 0.383, step = 0.001, width = 250)
    dpg.add_button(label="Done", callback=fun, tag ="button")
    dpg.add_text("")
    dpg.add_text("Flight Time:", tag="lmh")
    dpg.add_text("Flight Distance:", tag="lf")
    dpg.add_text("Maximum Flight Altitude:", tag="mh")
    dpg.add_text("Time To Reach Maximum Flight Altitude:", tag="tmh")
    
    dpg.bind_font(default_font)
    dpg.bind_item_font(b2, second_font)
    dpg.bind_item_font(b3, second_font)
    
    with dpg.plot(height=370, width=370, pos=(320,35), no_mouse_pos=True):
        dpg.add_plot_legend()
        dpg.add_plot_axis(dpg.mvXAxis, tag="x_axis")
        dpg.add_plot_axis(dpg.mvYAxis, tag="y_axis")

dpg.show_viewport()
dpg.start_dearpygui()
dpg.destroy_context()
